Lien du projet GitHub "__Yolov7__" : https://github.com/WongKinYiu/yolov7 </br>
Lien du dataset : https://www.kaggle.com/datasets/andrewmvd/hard-hat-detection

*Pour toute reproduction, veuillez avoir l'accord de l'équipe du cours "__Détection d'objets: l'intelligence artificielle au service de la sécurité__"*
***

# <center>TP : Projet "Safety First" 👷 - BricoLor x DetectAId</center>

***
### Enoncé : 
Notre entreprise "__DetecAId__" (se prononçant "detected" ou "detect aid"), propose des services de mise en place de solutions basées sur la technologie de la <ins>détection d'objets</ins> dans le cadre de la sécurité. 
Une entreprise du nom de __BricoLor__ (entreprise de construction), nous sollicite car ils ont été mobilisés pour la construction d'un stade dans le cadre du projet __JO2024__. 

Ils souhaiteraient mener à bien ce projet mobilisant plus de <ins>1000 employés</ins> (sur  le terrain), et ils ont décidé pour cela de mettre un point de priorité sur la sécurité, notamment sur le port des <ins>EPI</ins> (Equipement de protection Individuelle) pour les personnes se trouvant sur le chantier du futur stade.
***

In [3]:
#Verify your python version >=3.6 != 3.10 (3.9 recommended)
!py --version

Python 3.9.0


***
#### TODO 1 - Utilisation du modèle YOLOv7
__Tester la detection d'objet YOLOv7 sur une image et/ou vidéo__ - 2 points

In [5]:
#Write your code here
!py detect.py --weights yolov7.pt --conf 0.25 --img-size 640 --source inference/images/helmet.jpg

Namespace(weights=['yolov7.pt'], source='inference/images/helmet.jpg', img_size=640, conf_thres=0.25, iou_thres=0.45, device='', view_img=False, save_txt=False, save_conf=False, nosave=False, classes=None, agnostic_nms=False, augment=False, update=False, project='runs/detect', name='exp', exist_ok=False, no_trace=False)
Fusing layers... 
RepConv.fuse_repvgg_block
RepConv.fuse_repvgg_block
RepConv.fuse_repvgg_block
 Convert model to Traced-model... 
 traced_script_module saved! 
 model is traced! 

1 person, Done. (1298.5ms) Inference, (2.5ms) NMS
 The image with the result is saved in: runs\detect\exp8\helmet.jpg
Done. (1.351s)


YOLOR  v0.1-121-g2fdc7f1 torch 1.8.0+cpu CPU

Model Summary: 306 layers, 36905341 parameters, 6652669 gradients, 104.5 GFLOPS


Voici à quoi doit ressembler l'architecture du projet suite à l'ajout du dataset

```bash
yolov7 
└─── dataset
   │
   └─── images
            │
            └─── test
            │
            └─── train
            │
            └─── val
    └──  labels
            │
            └─── test
            │
            └─── train 
            │
            └─── val 
```

***
#### TODO 2 - Extraire les informations des annotations (fichier .xml)
__Remplir les lignes de codes manquantes__ - 8 points


In [16]:
import xml.etree.ElementTree as ET
import os 
from tqdm import tqdm
# Dictionary that maps class names to IDs
class_name_to_id_mapping = {"helmet": 0,
                           "person": 1,
                           "head": 2}

def extract_info_from_xml(xml_file):
    root = ET.parse(xml_file).getroot()
    
    # Initialise the info dict 
    info_dict = {}
    info_dict['bboxes'] = []

    #### TODO 2.1 - Parse the XML Tree ####
    #Write your code here
    for elem in root:
        
        #### TODO 2.2 - Get the file name ####
        #Write your code here  
        if elem.tag == "filename":
            info_dict['filename'] = elem.text
            
        #### TODO 2.3 - Get the image size ####
        #Write your code here 
        elif elem.tag == "size":
            image_size = []
            for subelem in elem:
                image_size.append(int(subelem.text))
            
            info_dict['image_size'] = tuple(image_size)
        
        #### TODO 2.4 - Get details of the bounding box ####
        #Write your code here 
        elif elem.tag == "object":
            bbox = {}
            for subelem in elem:
                if subelem.tag == "name":
                    bbox["class"] = subelem.text
                    
                elif subelem.tag == "bndbox":
                    for subsubelem in subelem:
                        bbox[subsubelem.tag] = int(subsubelem.text)            
            info_dict['bboxes'].append(bbox)
    
    return info_dict

***
#### Conversion des données sous le format YOLO

In [17]:
# Convert the info dict to the required yolo format and write it to disk
def convert_to_yolov7(info_dict):
    print_buffer = []
    
    # For each bounding box
    for b in info_dict["bboxes"]:
        try:
            class_id = class_name_to_id_mapping[b["class"]]
        except KeyError:
            print("Invalid Class. Must be one from ", class_name_to_id_mapping.keys())
        
        # Transform the bbox co-ordinates as per the format required by YOLO v5
        b_center_x = (b["xmin"] + b["xmax"]) / 2 
        b_center_y = (b["ymin"] + b["ymax"]) / 2
        b_width    = (b["xmax"] - b["xmin"])
        b_height   = (b["ymax"] - b["ymin"])
        
        # Normalise the co-ordinates by the dimensions of the image
        image_w, image_h, image_c = info_dict["image_size"]  
        b_center_x /= image_w 
        b_center_y /= image_h 
        b_width    /= image_w 
        b_height   /= image_h 
        
        #Write the bbox details to the file 
        print_buffer.append("{} {:.3f} {:.3f} {:.3f} {:.3f}".format(class_id, b_center_x, b_center_y, b_width, b_height))
        
    # Name of the file which we have to save 
    save_file_name = os.path.join("annotations", info_dict["filename"].replace("png", "txt"))
    
    # Save the annotation to disk
    print("\n".join(print_buffer), file= open(save_file_name, "w"))

annotations = [os.path.join('annotations', x) for x in os.listdir('annotations') if x[-3:] == "xml"]
annotations.sort()

# Convert and save the annotations
for ann in tqdm(annotations):
    info_dict = extract_info_from_xml(ann)
    convert_to_yolov7(info_dict)
annotations = [os.path.join('annotations', x) for x in os.listdir('annotations') if x[-3:] == "txt"]

100%|█████████████████████████████████████████████████████████████████████████████| 5000/5000 [00:23<00:00, 209.95it/s]


***
#### TODO 3 - Spliting du dataset
__Remplir les lignes manquantes__ - 4 points

In [18]:
import os
from sklearn.model_selection import train_test_split
import shutil

#### TODO 3.1 - Read images and annotations ####
#Write your code here
images = [os.path.join('images', x) for x in os.listdir('images')]
annotations = [os.path.join('annotations', x) for x in os.listdir('annotations') if x[-3:] == "txt"]

images.sort()
annotations.sort()

#### TODO 3.2 - Split the dataset into train-valid-test splits ####
#Write your code here
train_images, val_images, train_annotations, val_annotations = train_test_split(images, annotations, test_size = 0.2, random_state = 1)
val_images, test_images, val_annotations, test_annotations = train_test_split(val_images, val_annotations, test_size = 0.5, random_state = 1)

#Utility function to move images 
def move_files_to_folder(list_of_files, destination_folder):
    for f in list_of_files:
        try:
            shutil.move(f, destination_folder)
        except:
            print(f)
            assert False

#### TODO 3.3 - Move the splits into their folders ####
#Write your code here
move_files_to_folder(train_images, 'dataset/images/train/')
move_files_to_folder(val_images, 'dataset/images/val/')
move_files_to_folder(test_images, 'dataset/images/test/')
move_files_to_folder(train_annotations, 'dataset/labels/train/')
move_files_to_folder(val_annotations, 'dataset/labels/val/')
move_files_to_folder(test_annotations, 'dataset/labels/test/')

***
#### TODO 4 - Entraînement du modèle via transfert-learning
Exécuter la commande permettant d'entraîner le modèle sur le dataset - 2 points (+ points bonus)

Le choix des hyperparamètres depend de vous, des points bonus seront attribués selon le fine-tuning du modèle.

In [20]:
#Write your code here
!py train.py --workers 4 --img 640 --cfg cfg/training/yolov7-tiny.yaml --hyp data/hyp.scratch.p5.yaml --batch 16 --epochs 50 --data data/safety_helmet_data.yaml --weights yolov7-tiny.pt --name yolo_safety_helmet_detection_50_epochs

Traceback (most recent call last):
  File "C:\Users\Maathess\Desktop\TP_Safety_First\yolov7\train.py", line 21, in <module>
    from torch.utils.tensorboard import SummaryWriter
  File "C:\Users\Maathess\Desktop\TP_Safety_First\yolov7\venv\lib\site-packages\torch\utils\tensorboard\__init__.py", line 8, in <module>
    from .writer import FileWriter, SummaryWriter  # noqa F401
  File "C:\Users\Maathess\Desktop\TP_Safety_First\yolov7\venv\lib\site-packages\torch\utils\tensorboard\writer.py", line 9, in <module>
    from tensorboard.compat.proto.event_pb2 import SessionLog
  File "C:\Users\Maathess\Desktop\TP_Safety_First\yolov7\venv\lib\site-packages\tensorboard\compat\proto\event_pb2.py", line 17, in <module>
    from tensorboard.compat.proto import summary_pb2 as tensorboard_dot_compat_dot_proto_dot_summary__pb2
  File "C:\Users\Maathess\Desktop\TP_Safety_First\yolov7\venv\lib\site-packages\tensorboard\compat\proto\summary_pb2.py", line 17, in <module>
    from tensorboard.compat.proto

***
#### TODO 5 - Inférence sur le modèle entraîné 
Exécuter la commande permettant de réaliser une détection sur une image et/ou vidéo à l'aide du modèle entraîné ci-dessus et afficher les résultats dans le notebook et les interpréter - 2 points 

In [8]:
#Write your code here - image
!py detect.py --weights runs/train/yolo_safety_helmet_detection_50_epochs/weights/best.pt --conf 0.25 --img-size 640 --source inference/images/helmet_test.jpg

Namespace(weights=['runs/train/yolo_safety_helmet_detection_50_epochs/weights/best.pt'], source='inference/images/helmet_test.jpg', img_size=640, conf_thres=0.25, iou_thres=0.45, device='', view_img=False, save_txt=False, save_conf=False, nosave=False, classes=None, agnostic_nms=False, augment=False, update=False, project='runs/detect', name='exp', exist_ok=False, no_trace=False)
Fusing layers... 
IDetect.fuse
 Convert model to Traced-model... 
 traced_script_module saved! 
 model is traced! 

1 helmet, Done. (172.5ms) Inference, (0.5ms) NMS
 The image with the result is saved in: runs\detect\exp11\helmet_test.jpg
Done. (0.248s)


YOLOR  v0.1-121-g2fdc7f1 torch 1.8.0+cpu CPU

Model Summary: 208 layers, 6013008 parameters, 0 gradients, 13.0 GFLOPS


In [7]:
#Write your code here - vidéo
!py detect.py --weights runs/train/yolo_safety_helmet_detection_50_epochs/weights/best.pt --conf 0.25 --img-size 640 --source inference/videos/helmet_3.mp4

Namespace(weights=['runs/train/yolo_safety_helmet_detection_50_epochs/weights/best.pt'], source='inference/videos/helmet_3.mp4', img_size=640, conf_thres=0.25, iou_thres=0.45, device='', view_img=False, save_txt=False, save_conf=False, nosave=False, classes=None, agnostic_nms=False, augment=False, update=False, project='runs/detect', name='exp', exist_ok=False, no_trace=False)
Fusing layers... 
IDetect.fuse
 Convert model to Traced-model... 
 traced_script_module saved! 
 model is traced! 

video 1/1 (1/359) C:\Users\Maathess\Desktop\TP_Safety_First\yolov7\inference\videos\helmet_3.mp4: 1 head, Done. (184.5ms) Inference, (0.5ms) NMS
video 1/1 (2/359) C:\Users\Maathess\Desktop\TP_Safety_First\yolov7\inference\videos\helmet_3.mp4: 1 head, Done. (104.0ms) Inference, (0.6ms) NMS
video 1/1 (3/359) C:\Users\Maathess\Desktop\TP_Safety_First\yolov7\inference\videos\helmet_3.mp4: 1 helmet, 1 head, Done. (104.0ms) Inference, (0.5ms) NMS
video 1/1 (4/359) C:\Users\Maathess\Desktop\TP_Safety_First

YOLOR  v0.1-121-g2fdc7f1 torch 1.8.0+cpu CPU

Model Summary: 208 layers, 6013008 parameters, 0 gradients, 13.0 GFLOPS


***
#### TODO 6 - Inférence sur le modèle entraîné 
Afficher les résultat de l'entraînement - 2 points 

In [ ]:
#Write your code here